In [1]:
from getpass import getpass
import os
import time

import pandas as pd
import requests

import kdbai_client as kdbai

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import KDBAI

In [2]:
KDBAI_ENDPOINT = input('KDB.AI endpoint: ')
KDBAI_API_KEY = getpass('KDB.AI API key: ')
os.environ['OPENAI_API_KEY'] = getpass('OpenAI API Key: ')

KDB.AI endpoint:  https://ui.qa.cld.kx.com/instance/wpxy4fhbtr
KDB.AI API key:  ········
OpenAI API Key:  ········


In [3]:
TEMP = 0.0
K = 3

In [4]:
print('Create KDB.AI session...')
session = kdbai.Session(endpoint=KDBAI_ENDPOINT, api_key=KDBAI_API_KEY)

Create KDB.AI session...


In [5]:
print('Create table "documents"...')
schema = {'columns': [{'name': 'id', 'pytype': 'str'},
                      {'name': 'text', 'pytype': 'bytes'},
                      {'name': 'embeddings',
                               'pytype': 'float32',
                               'vectorIndex': {'dims': 1536, 'metric': 'L2', 'type': 'hnsw'}},
                      {'name': 'tag', 'pytype': 'str'},
                      {'name': 'title', 'pytype': 'bytes'}]}
table = session.create_table('documents', schema)

Create table "documents"...


In [6]:
%%time
URL = 'https://www.conseil-constitutionnel.fr/node/3850/pdf'
PDF = 'Déclaration_des_droits_de_l_homme_et_du_citoyen.pdf'
open(PDF, 'wb').write(requests.get(URL).content)

CPU times: user 42 ms, sys: 6.25 ms, total: 48.2 ms
Wall time: 222 ms


562978

In [7]:
%%time
print('Read PDF...')
loader = PyPDFLoader(PDF)
pages = loader.load_and_split()
len(pages)

Read PDF...
CPU times: user 130 ms, sys: 12.8 ms, total: 143 ms
Wall time: 146 ms


3

In [8]:
%%time
print('Create Vector DB from PDF text...')
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')
texts = [p.page_content for p in pages]
metadata = pd.DataFrame(index=list(range(len(texts))))
metadata['tag'] = 'law'
metadata['title'] = 'Déclaration des Droits de l\'Homme et du Citoyen de 1789'.encode('utf-8')
vectordb = KDBAI.from_texts(session, 'documents', texts=texts, embedding=embeddings, metadata=metadata)

Create Vector DB from PDF text...
CPU times: user 285 ms, sys: 30 ms, total: 315 ms
Wall time: 1.98 s


In [9]:
%%time
print('Create LangChain pipeline...')
qabot = RetrievalQA.from_chain_type(chain_type='stuff',
                                    llm=ChatOpenAI(model='gpt-3.5-turbo-16k', temperature=TEMP), 
                                    retriever=vectordb.as_retriever(search_kwargs=dict(k=K)),
                                    return_source_documents=True)

Create LangChain pipeline...
CPU times: user 626 µs, sys: 7 µs, total: 633 µs
Wall time: 634 µs


In [10]:
%%time
Q = 'Summarize the document in English:'
print(f'\n\n{Q}\n')
print(qabot(dict(query=Q))['result'])



Summarize the document in English:

The document is the Declaration of the Rights of Man and of the Citizen of 1789. It was written by the representatives of the French people and aims to declare the natural, inalienable, and sacred rights of every individual. These rights include freedom, property, security, and resistance to oppression. The document emphasizes the importance of equality and the principle that sovereignty resides in the nation. It also highlights the role of law in protecting individual rights and ensuring the common good. The document asserts the right to freedom of thought, expression, and communication, as well as the right to a fair trial and the presumption of innocence. It emphasizes the importance of public accountability and the need for a just and equitable distribution of public contributions. The document concludes by stating that any society without the guarantee of rights and the separation of powers does not have a constitution. It also recognizes the 

In [11]:
%%time
Q = 'Is it a fair law and why ?'
print(f'\n\n{Q}\n')
print(qabot(dict(query=Q))['result'])



Is it a fair law and why ?

As an AI language model, I don't have personal opinions. However, I can provide some analysis. The text provided is the Declaration of the Rights of Man and of the Citizen of 1789, which is considered a foundational document in the history of human rights. It asserts the principles of equality, freedom, and justice. Many consider it to be a fair law because it recognizes the inherent rights and freedoms of individuals and establishes the principles of a just society. However, the interpretation and application of these rights can vary, and opinions on the fairness of specific laws may differ.
CPU times: user 70.3 ms, sys: 6.44 ms, total: 76.7 ms
Wall time: 9.65 s


In [12]:
%%time
Q = 'What are the rights and duties of the man, the citizen and the society ?'
print(f'\n\n{Q}\n')
print(qabot(dict(query=Q))['result'])



What are the rights and duties of the man, the citizen and the society ?

According to the Declaration of the Rights of Man and of the Citizen of 1789, the rights and duties of man, citizen, and society are as follows:

Rights of Man:
1. Men are born and remain free and equal in rights. Social distinctions can only be based on common utility.
2. The purpose of political association is to protect the natural and imprescriptible rights of man, which include liberty, property, security, and resistance to oppression.
3. Sovereignty resides essentially in the nation. No body or individual can exercise authority that does not emanate expressly from it.
4. Freedom consists of being able to do anything that does not harm others. The exercise of natural rights of each man has no limits other than those that ensure the enjoyment of these same rights by other members of society. These limits can only be determined by law.
5. The law has the right to prohibit only actions harmful to society. Any

In [13]:
%%time
Q = 'Is this law practical ?'
print(f'\n\n{Q}\n')
print(qabot(dict(query=Q))['result'])



Is this law practical ?

As an AI language model, I don't have personal opinions. Whether a law is practical or not is subjective and can depend on various factors such as the context, societal norms, and individual perspectives. It is up to legal experts, policymakers, and society as a whole to assess the practicality and effectiveness of a law based on its implementation and impact.
CPU times: user 54.9 ms, sys: 7.52 ms, total: 62.4 ms
Wall time: 4.54 s


In [14]:
# Clean up KDB.AI "documents" table and index for similarity search
# so this notebook could be played again and again
session.table('documents').drop()

True